 # Volatility Spillover Analysis Using VAR on PIT-Transformed Covariances



 This notebook loads the PIT-transformed vectorized realized covariances (vech format),

 fits a VAR model, and computes the generalized forecast error variance decomposition (GVD),

 including both variances and covariances in the spillover structure.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown


 ## Step 1: Select and Load PIT-transformed vech Dataset

In [ ]:
option_selector = widgets.Dropdown(
    options=['europe', 'australia'],
    value='europe',
    description='Dataset:',
)
button = widgets.Button(description="Load Dataset", button_style='success', icon='check')

def on_button_click(b):
    global pit_vech
    clear_output(wait=True)
    display(option_selector, button)
    selected_option = option_selector.value
    file_path = f"parquet_files/pit_transformed_vech_{selected_option}.parquet"
    try:
        pit_vech = pd.read_parquet(file_path)
        print(f"✅ Loaded: {file_path}")
        display(pit_vech.head())
    except Exception as e:
        print(f"❌ Failed to load: {e}")

button.on_click(on_button_click)
display(option_selector, button)


 ## Step 2: Fit VAR(p) Model

In [ ]:
p = 30
model = VAR(pit_vech)
results = model.fit(p)
Phi = results.coefs  # shape (p, N, N)
Sigma = results.sigma_u  # residual covariance matrix
N = Sigma.shape[0]
H = 1


 ## Step 3: Compute Moving Average Coefficients A_h

In [ ]:
A = [np.eye(N)]
for h in range(1, H):
    A_h = np.zeros((N, N))
    for j in range(1, min(h, p)+1):
        A_h += Phi[j-1] @ A[h-j]
    A.append(A_h)


 ## Step 4: Compute Generalized Variance Decomposition (GVD)

In [ ]:
theta_g = np.zeros((N, N))
sigma_diag_inv = np.diag(1 / np.diag(Sigma))

for i in range(N):
    e_i = np.zeros(N)
    e_i[i] = 1
    denom = sum([e_i @ A[h] @ Sigma @ A[h].T @ e_i for h in range(H)])
    for j in range(N):
        e_j = np.zeros(N)
        e_j[j] = 1
        numer = sum([(e_i @ A[h] @ Sigma @ e_j)**2 for h in range(H)])
        theta_g[i, j] = sigma_diag_inv[j, j] * numer / denom

# Normalize row-wise
theta_g_normalized = theta_g / theta_g.sum(axis=1, keepdims=True)


 ## Step 5: Compute Total Spillover Index (TSI)

In [ ]:
TSI = 100 * (np.sum(theta_g_normalized) - np.trace(theta_g_normalized)) / N
print(f"Total Spillover Index (TSI) at horizon {H}: {TSI:.2f}%")




 ## Step 6: Compute Directional and Net Spillovers

In [ ]:
# Convert GVD matrix to percent
spillover_matrix = theta_g_normalized * 100

# Directional spillovers
directional_to = spillover_matrix.sum(axis=1) - np.diag(spillover_matrix)
directional_from = spillover_matrix.sum(axis=0) - np.diag(spillover_matrix)
net_directional = directional_to - directional_from

# Construct spillover table
spillover_table = pd.DataFrame(spillover_matrix,
                               index=pit_vech.columns,
                               columns=pit_vech.columns)
spillover_table["Directional FROM others"] = directional_from
spillover_table.loc["Directional TO others"] = list(directional_to) + [directional_to.sum()]
spillover_table.loc["NET Directional"] = list(net_directional) + [np.nan]


 ## Step 7: Save Spillover Table

In [ ]:
spillover_table.to_parquet(f"parquet_files/volatility_spillovers_VAR_fullcov_{option_selector.value}.parquet")


 ## Step 8: Display Final Spillover Table with Caption

In [ ]:
display_table = spillover_table.round(2)

try:
    caption = f"""
**Table X**: Spillovers based on VAR-GVD using PIT-transformed realized covariances (vech format).

This table reports volatility spillovers across markets using a one-day forecast horizon VAR model fitted on vectorized PIT-transformed covariance matrices.
The $ij$ element reflects the contribution of market $j$'s shocks to the forecast error variance of market $i$.
Directional spillovers and Total Spillover Index (TSI) are computed as in Diebold and Yilmaz (2012).

**TSI = {TSI:.2f}%**
"""
    display(display_table)
    display(Markdown(caption))
except Exception as e:
    print("Failed to display table:", e)
    print(display_table)
    print(f"TSI = {TSI:.2f}%")
